In [4]:
import pandas as pd
import glob
import numpy as np
import matplotlib.pyplot as plt

In [6]:
# Import data

path =r'D:\data_raw' # Please put all .csv documents into one folder
company_list = ['AAPL', 'AXP', 'BA', 'CAT', 'CSCO', 'CVX', 'DIS', 'DWDP', 'GE', 'GS', \
                           'HD', 'IBM', 'INTC', 'JNJ', 'JPM', 'KO', 'MCD', 'MMM', 'MRK', \
                            'MSFT', 'NKE', 'PFE', 'PG', 'TRV', 'UNH', 'UTX', 'V', 'VZ', 'WMT', 'XOM' ] # Input company name as a list
#company_list = ['AAPL', 'AXP']
company_data = {} # Store company data as a dictionary: {company_name1: data1, company_name2: data2, ...}
for company in company_list:
    frame = pd.DataFrame()
    list_ = []
    allFiles = glob.glob(path + '/' +company + '*.csv')
    for file_ in allFiles:
        df = pd.read_csv(file_, index_col=None, header=0)
        list_.append(df)
    frame = pd.concat(list_)
    company_data[company] = frame

In [9]:
# Data cleaning

for company in company_data.keys():
    company_data[company].drop(company_data[company].columns[2:], axis = 1, inplace = True) # Keep only ''Time interval'  and 'Close' price
    company_data[company].drop(company_data[company][company_data[company]['Time Interval'] =='Summary'].index, axis = 0, inplace = True) # Delete Summary rows
    company_data[company]['Log_Price'] = np.log(company_data[company]['Close']) # Compute log price
    company_data[company]['Log_Return'] = company_data[company]['Log_Price'].diff() # Compute log return r
    company_data[company].dropna(axis = 0, how = 'any', inplace = True) # Drop lines with empty data

In [10]:
# Reshape data using 'Time Interval ' as columns

reshaped_data = {}
for company in company_data.keys():
    reshaped_data[company] = pd.DataFrame()
    group_by_5 = company_data[company].groupby(by = 'Time Interval') # Group data by 5 min time interval
    for i_5, i_5_data in group_by_5:
        reshaped_data[company][i_5] = pd.Series(list(i_5_data['Log_Return']))

In [11]:
# Compute daily return, RDVar, RDSkew and RDKurtosis

for com in reshaped_data.keys():
    N = reshaped_data[com].shape[1]
    reshaped_data[com]['DR'] = reshaped_data[com].apply(lambda x: np.sum(x[:N]), axis = 1)
    reshaped_data[com]['RDVar'] = reshaped_data[com].apply(lambda x: np.sum(np.square(x[:N])), axis = 1)
    reshaped_data[com]['RDSkew'] = reshaped_data[com].apply(lambda x: np.sqrt(N)*np.sum(np.power(x[:N], 3))/(x['RDVar']**1.5), axis = 1)
    reshaped_data[com]['RDKurtosis'] = reshaped_data[com].apply(lambda x: N*np.sum(np.power(x[:N], 4))/(x['RDVar']**2), axis = 1)

In [12]:
reshaped_data['AAPL'].head()

,09:35 - 09:40,09:40 - 09:45,09:45 - 09:50,09:50 - 09:55,09:55 - 10:00,10:00 - 10:05,10:05 - 10:10,10:10 - 10:15,10:15 - 10:20,10:20 - 10:25,...,15:30 - 15:35,15:35 - 15:40,15:40 - 15:45,15:45 - 15:50,15:50 - 15:55,15:55 - 16:00,DR,RDVar,RDSkew,RDKurtosis
0,0.002817,-0.000975,-0.001386,0.000716,0.001755,-0.000910,-0.000976,-0.000651,0.000651,-0.000195,...,0.000065,0.000716,0.000033,0.001008,-0.000143,-0.000377,0.002083,0.000049,-0.034297,5.451433
1,0.000970,-0.001746,0.001811,-0.002226,-0.000557,0.001554,-0.000906,-0.002075,0.001038,0.000583,...,0.000129,0.000259,0.000000,-0.000971,-0.000389,0.000777,-0.000647,0.000039,-0.493605,4.409002
2,-0.000521,-0.001434,0.000228,-0.000555,0.001988,-0.000287,-0.001278,0.000033,-0.000750,-0.000882,...,0.000033,0.000587,-0.000130,0.000847,-0.000723,0.000339,-0.000840,0.000033,0.197202,3.409307
3,-0.000310,0.001020,0.001547,0.000419,-0.002095,0.000581,0.000677,0.000387,-0.000161,-0.000354,...,0.000412,0.000425,-0.000257,-0.000097,-0.000322,0.000740,0.003287,0.000033,-0.399728,3.941852
4,-0.000387,-0.001677,-0.000194,-0.000588,-0.000123,0.001227,0.000452,-0.000129,-0.001065,0.001065,...,-0.000258,0.000355,0.000032,0.000387,0.000258,0.000644,0.000580,0.000024,-0.041219,3.185754


In [13]:
# Construct weekly data (5 trading days a week)
weekly_data = {}
for com in reshaped_data.keys():
    weekly_data[com] = pd.DataFrame()
    weekly_data[com]['WR'] = reshaped_data[com]['DR'].rolling(5).sum()[4::5]
    weekly_data[com]['RVol'] = np.sqrt(252.*reshaped_data[com]['RDVar'].rolling(5).mean()[4::5])
    weekly_data[com]['RSkew'] = reshaped_data[com]['RDSkew'].rolling(5).mean()[4::5]
    weekly_data[com]['RKurt'] = reshaped_data[com]['RDKurtosis'].rolling(5).mean()[4::5]

In [14]:
weekly_data['AAPL'].head()

,WR,RVol,RSkew,RKurt
4,0.004462,0.094435,-0.154330,4.079470
9,0.000635,0.097790,0.054424,5.149241
14,-0.001088,0.103214,-0.197347,5.088467
19,0.013911,0.118723,0.149588,3.637123
24,0.007153,0.187124,-0.121788,5.357063


In [15]:
weekly_data['AAPL'].shape

(251, 4)

In [16]:
weekly_data['AXP'].head()

,WR,RVol,RSkew,RKurt
4,0.002398,0.095343,-0.146904,5.640521
9,0.001399,0.093984,0.294513,5.502431
14,0.002553,0.158847,-0.781136,10.384099
19,0.022534,0.105096,0.465386,5.256560
24,0.005436,0.112088,-0.068529,4.600702
